## StarTracker Fast Camera rotation

Craig Lage - 17-Apr-24

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
from scipy.ndimage import median_filter
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import pandas as pd
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.geom import Point2D, SpherePoint, Angle, AngleUnit

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

### Next solve the fast image

In [ ]:
# Code from Merlin Fisher-Levine to solve fast camera images
def runCharactierizeImage(exp, snr, minPix):
    """Run the image characterization task, finding only bright sources.

    Parameters
    ----------
    exp : `lsst.afw.image.Exposure`
        The exposure to characterize.
    snr : `float`
        The SNR threshold for detection.
    minPix : `int`
        The minimum number of pixels to count as a source.

    Returns
    -------
    result : `lsst.pipe.base.Struct`
        The result from the image characterization task.
    """
    charConfig = CharacterizeImageConfig()
    charConfig.doMeasurePsf = False
    charConfig.doApCorr = False
    charConfig.doDeblend = False
    charConfig.repair.doCosmicRay = False

    charConfig.detection.minPixels = minPix
    charConfig.detection.thresholdValue = snr
    charConfig.detection.includeThresholdMultiplier = 1
    charConfig.detection.nSigmaToGrow = 0

    charConfig.psfIterations = 1
    charConfig.installSimplePsf.fwhm = 5
    charConfig.installSimplePsf.width = 51
    
    # fit background with the most simple thing possible as we don't need
    # much sophistication here. weighting=False is *required* for very
    # large binSizes.
#     charConfig.background.algorithm = 'AKIMA'
    charConfig.background.approxOrderX = 1
    charConfig.background.approxOrderY = -1
    charConfig.background.binSize = max(exp.getWidth(), exp.getHeight())
    charConfig.background.weighting = False

    # set this to use all the same minimal settings as those above
    charConfig.detection.background = charConfig.background

    charTask = CharacterizeImageTask(config=charConfig)

    charResult = charTask.run(exp)
    return charResult

def solveFastCamera(filename, doPlot=False):
    exp = starTrackerFileToExposure(filename)
    exp.image.array = median_filter(exp.image.array, 2, mode='reflect')
    result = runCharactierizeImage(exp, 5, 25)
    sources = filterSourceCatOnBrightest(result.sourceCat, .5, minSources=5)
    if doPlot:
        plot(exp, sources, doSmooth=False)
    result = solver.run(exp, sources, True, useGaia=True, percentageScaleError=10, radius=5)
    if result is not None:
        fastWcs = result.wcs
        fastVisitInfo = exp.visitInfo
    else:
        print('fit failed')
        return None
    return fastWcs

In [ ]:
#Enter which image you want to look at
[camera, num] = ['Fast', 103]
dayObs = 20240417
year = int(dayObs/10000)
month = int((dayObs - 10000 * year)/100)
day = int((dayObs - 10000 * year - 100 * month))
seqNum = 1108
path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
print(path)
filename = path + f"GC{num}_O_{dayObs}_{seqNum:06}.fits"
print(filename)
fast_exp = starTrackerFileToExposure(filename)
result = solveFastCamera(filename, doPlot=False)
fast_wcs = result

# This is a back-up
#from lsst.afw.geom import SkyWcs
#fast_wcs = SkyWcs.readFits('/home/mfisherlevine/temp/fastCamFits/GC103_O_20230321_001160_wcs.fits')

## Now calculate the rotation and plot the results

In [ ]:
def alt_az_arrows(fast_wcs, fast_exp):
    # This calculates the Alt Az directions
    mData = fast_exp.getInfo().getMetadata()
    time = Time((Time(mData["DATE-BEG"], scale='tai').unix_tai + Time(mData["DATE-END"], scale='tai').unix_tai)/2.0, \
                format='unix_tai', scale='tai')
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
    center = np.array((int(fast_exp.image.array.shape[1] / 2), int(fast_exp.image.array.shape[0] / 2)))
    center_raDec = fast_wcs.pixelToSky(center[0], center[1])
    skyLocation = SkyCoord(center_raDec.getRa().asDegrees() * u.deg, center_raDec.getDec().asDegrees() * u.deg)
    obsAltAz = skyLocation.transform_to(altAz)
    az = obsAltAz.az.deg
    alt = obsAltAz.alt.deg
    azPlus = az + 50.0 / 3600.0 / np.cos(obsAltAz.alt.rad) # add 500 arcseconds
    altPlus = alt + 50.0 / 3600.0 # add 500 arcseconds
    altAzPlus = SkyCoord(AltAz(alt=alt*u.deg, az=azPlus*u.deg, obstime=time, location=SIMONYI_LOCATION))
    RaDecAzPlus = altAzPlus.transform_to(ICRS)
    SpherePointAzPlus = SpherePoint(Angle(RaDecAzPlus.ra.rad), Angle(RaDecAzPlus.dec.rad))
    azPlusPixels = fast_wcs.skyToPixel(SpherePointAzPlus)
    altPlusAz = SkyCoord(AltAz(alt=altPlus*u.deg, az=az*u.deg, obstime=time, location=SIMONYI_LOCATION))
    RaDecAltPlus = altPlusAz.transform_to(ICRS)
    SpherePointAltPlus = SpherePoint(Angle(RaDecAltPlus.ra.rad), Angle(RaDecAltPlus.dec.rad))
    altPlusPixels = fast_wcs.skyToPixel(SpherePointAltPlus)
    azPlusArrowLength = np.array(azPlusPixels) - np.array(center)
    altPlusArrowLength = np.array(altPlusPixels) - np.array(center)
    return [center, azPlusArrowLength, altPlusArrowLength]


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,8))
ax.set_title(f"StarTracker Fast, 20240417 SeqNum {seqNum}")
ax.imshow(fast_exp.image.array,  interpolation='Nearest', cmap='gray', vmin=10, vmax=100, origin='lower')
[center, azPlusArrowLength, altPlusArrowLength] = alt_az_arrows(fast_wcs, fast_exp)
ax.arrow(center[0], center[1], azPlusArrowLength[0], azPlusArrowLength[1], color='lightgreen', \
        head_width=5, head_length=10, lw = 2)
ax.text(center[0] + azPlusArrowLength[0]+5, center[1] + azPlusArrowLength[1]-10, \
        'Plus AzCosAlt\n50"', color='lightgreen', fontsize=18)
ax.arrow(center[0], center[1], altPlusArrowLength[0], altPlusArrowLength[1], color='lightgreen', \
        head_width=5, head_length=10, lw = 2)
ax.text(center[0] + altPlusArrowLength[0]+10, center[1] + altPlusArrowLength[1], \
        'Plus Alt\n50"', color='lightgreen', fontsize=18)
# Calculate angle from vertical to +Alt
altAngle = 90.0 - np.arctan(altPlusArrowLength[1] / altPlusArrowLength[0]) * 180.0 / np.pi
ax.text(100, 100, \
        f"Angle from vertical to +Alt = {altAngle:.2f} degrees", color='lightgreen', fontsize=18)

plt.savefig(F"/home/cslage/DATA/Fast_Camera_Axes_{dayObs}_{seqNum}.png")

In [ ]:
fast_wcs.getPixelScale